In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import csv
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

In [ ]:
model = tf.keras.models.load_model('/content/model.h5')

In [ ]:
def load_video(path, max_frames=30, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []

    try:
        for i in range(max_frames):
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, resize)
            frames.append(frame/255)

    finally:
        cap.release()

    return frames

In [ ]:
import cv2
import os

def save_video_segments(video_path, output_dir, frames_per_segment=30):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Calculate the number of segments
    num_segments = total_frames // frames_per_segment

    frame_count = 0

    # Read and save video segments
    for i in range(num_segments):
        frames = []
        for j in range(frames_per_segment):
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)

        # Check if we have enough frames for a segment
        if len(frames) == frames_per_segment:
            # Create a VideoWriter object for the segment
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            output_path = os.path.join(output_dir, f'segment_{i+1}.mp4')
            out = cv2.VideoWriter(output_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

            # Write each frame to the video file
            for frame in frames:
                out.write(frame)

            # Release the VideoWriter
            out.release()

    # Release the VideoCapture
    cap.release()

# Example usage
video_path = '/content/robbery.mp4'
output_directory = '/content/segment'
save_video_segments(video_path, output_directory)


In [ ]:
from tensorflow.keras.utils import Sequence

class VideoDataGenerator(Sequence):
    def __init__(self, video_paths, labels, batch_size, frame_shape=(224, 224), shuffle=True):
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.frame_shape = frame_shape
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.video_paths) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = []

        batch_data = []

        for i in indices:
            frames = self.load_and_preprocess_video(self.video_paths[i])
            if(len(frames) == 30):
                batch_data.append(frames)
                batch_labels.append(self.labels[i])

        return np.array(batch_data), np.array(batch_labels)

    def on_epoch_end(self):
        self.indices = np.arange(len(self.video_paths))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def load_and_preprocess_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, self.frame_shape)
            frames.append(frame/255)
        cap.release()
        return np.array(frames[:30])

video_paths = ['/content/segments/segment_{}.mp4'.format(i) for i in range(1, 17)]
video_labels = [1 for i in range(16)]
batch_size = 2

video_generator = VideoDataGenerator(video_paths, video_labels, batch_size)


In [ ]:
len(frames)

30

In [ ]:
prediction = model.predict(video_generator)

8/8 [==============================] - 35s 4s/step


In [ ]:
prediction

array([[2.24222153e-04, 3.40225961e-05, 6.84058468e-04, 2.19395082e-03,
        3.47813852e-02, 1.75224704e-05, 1.19122515e-05, 8.82567465e-01,
        1.06997555e-03, 6.28492460e-02, 1.04065123e-03, 6.18816295e-04,
        1.33615956e-02, 5.45095070e-04],
       [1.99848655e-04, 3.23876811e-05, 8.03969684e-04, 8.04600539e-04,
        1.41205126e-03, 2.74694557e-05, 4.07141633e-05, 5.97266138e-01,
        2.50639860e-04, 6.46675676e-02, 1.03982922e-04, 3.93103168e-04,
        3.33821118e-01, 1.76362781e-04],
       [4.07243118e-04, 2.64854752e-05, 3.26124631e-04, 2.96223414e-04,
        3.33619281e-03, 3.98586126e-05, 3.76644930e-05, 8.46745193e-01,
        1.79091905e-04, 4.73360047e-02, 2.62608868e-04, 3.81378137e-04,
        1.00324742e-01, 3.01172462e-04],
       [5.21958980e-04, 2.73891874e-06, 6.69467612e-04, 5.52367140e-03,
        8.35453067e-03, 2.12909290e-04, 2.61228997e-05, 7.75075197e-01,
        1.70788250e-03, 1.89956114e-01, 8.74346064e-04, 6.53293042e-04,
        1.568

In [ ]:
column_averages = np.mean(prediction, axis=0)

print(column_averages)

[5.6630705e-04 6.5027183e-05 1.1887921e-03 2.1362442e-03 7.3188632e-03
 2.2123150e-04 4.1088389e-04 6.0542059e-01 8.1641483e-04 2.2558886e-01
 7.7844004e-04 4.7926200e-04 1.5424687e-01 7.6212944e-04]
